In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input, Lambda
from keras.layers.wrappers import Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD, Adagrad
from keras.layers.convolutional import Conv1D
from keras.layers import SpatialDropout1D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.layers import concatenate,dot
from keras.preprocessing import sequence
import keras.backend as K
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pandas as pd
import numpy as np

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [20]:
#embed_size = 50 # how big is each word vector
max_features = 30000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use
train = pd.read_csv("train_stance_pre.csv")
train_body = pd.read_csv("train_body_pre.csv")
test = pd.read_csv("test_stance_pre.csv")
test_body = pd.read_csv("test_body_pre.csv")
list_sentences_train = train["Headline"].fillna("_na_").values
list_sentences_body = train_body["articleBody"]
list_sentences_test = test["Headline"].fillna("_na_").values
list_sentences_body1 = test_body["articleBody"]
#subm = pd.read_csv("sample_submission.csv")
#cat = pd.read_json("categories.json")
list_classes = ["agree","disagree","discuss","unrelated"]

#list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
tokenizer.fit_on_texts(list(list_sentences_body))
tokenizer.fit_on_texts(list(list_sentences_test))
tokenizer.fit_on_texts(list(list_sentences_body1))
list_tokenized_train_sent = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_train_body = tokenizer.texts_to_sequences(list_sentences_body)
list_tokenized_test_sent = tokenizer.texts_to_sequences(list_sentences_test)
list_tokenized_test_body = tokenizer.texts_to_sequences(list_sentences_body1)
X_t = pad_sequences(list_tokenized_train_sent, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_train_body, maxlen=300)
X_tte = np.zeros((len(X_t),300),dtype = np.int32)
for i in range(len(train['Body ID'])):
    X_tte[i] = X_te[train_body['Body ID']==train['Body ID'][i]]
train_text = X_t
train_body = X_tte
y_train = np.array(y,dtype=np.float32)
y = test[list_classes].values
X_t = pad_sequences(list_tokenized_test_sent, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test_body, maxlen=300)
X_tte = np.zeros((len(X_t),300),dtype = np.int32)
for i in range(len(test['Body ID'])):
    X_tte[i] = X_te[test_body['Body ID']==test['Body ID'][i]]
test_text = X_t
test_body = X_tte
y_test = np.array(y,dtype=np.float32)
#[test_text,test_body],y_test

In [ ]:
len(tokenizer.word_index)

In [ ]:
#label_3_idx = sum([np.where(y==1)[1] == 3])
#label_2_idx = sum([np.where(y==1)[1] == 2])
#label_1_idx = sum([np.where(y==1)[1] == 1])
#label_0_idx = sum([np.where(y==1)[1] == 0])
#label_0_idx = [idx  for idx,t in enumerate(label_0_idx) if t == 1]
#label_1_idx = [idx  for idx,t in enumerate(label_1_idx) if t == 1]
#label_2_idx = [idx  for idx,t in enumerate(label_2_idx) if t == 1]
#label_3_idx = [idx  for idx,t in enumerate(label_3_idx) if t == 1]
#np.sum(label_1_idx)

In [ ]:
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


def report_score(actual,predicted):
    score,cm = score_submission(actual,predicted)
    best_score, _ = score_submission(actual,actual)

    print_confusion_matrix(cm)
    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score

In [ ]:
category_dict = {'unrelated': 3 , 'agree':0, 'disagree':1, 'discuss':2}
def evaluate_answer(model,model_inp, true):
    inv_category_dict = {3:'unrelated', 0: 'agree', 1: 'disagree', 2: 'discuss'}
    predicted = model.predict(model_inp)
    predicted = np.argmax(predicted,axis = 1)
    t = np.argmax(true,axis = 1)
    ground = list()
    pred = list()
    for i in predicted:
        pred.append(inv_category_dict[i])
    for i in t:
        ground.append(inv_category_dict[i])
    report_score(ground, pred)

In [ ]:
from gensim import models
embed_size = 300 # how big is each word vector
#max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
max_features = min(len(tokenizer.word_index),30000)
maxlen = 100 # max number of words in a comment to use
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(max_features, embed_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim
def loadGloveModel():
    print("Loading Glove Model")

    f = open('glove.6B.100d.txt','r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel
def loadGoogleModel():
    w = models.KeyedVectors.load_word2vec_format(
    './GoogleNews-vectors-negative300.bin', binary=True)
    return w
def get_embed(word_idx,max_features,embed_dim):
    #glove = loadGloveModel()
    glove = loadGoogleModel()
    #all_embs = np.stack(glove.to_numpy())
    #all_embs = np.stack(glove.values())
    #emb_mean,emb_std = all_embs.mean(), all_embs.std()
    emb_mean = 0
    emb_std = 0.6
    word_index = word_idx
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features+1, embed_dim))
    #print("go into model")
    for word, i in word_index.items():
        if i >= max_features: continue
        try: 
            embedding_vector = glove[word]
        except:
            None
            #print('skip')
            #embedding_vector = np.random.normal(scale=0.6, size=(100, ))
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
embedding_matrix = get_embed(tokenizer.word_index,max_features,embed_size)

In [ ]:
'''category_dict = {'unrelated': 0 , 'agree':1, 'disagree':2, 'discuss':3}
def create_dataset(data,ids,isTest,t,max_head,max_body):
    
    from keras.utils.np_utils import to_categorical
    head = list()
    body = list()
    y = list()
    NUM_CLASSES = 4
    category_dict = {'unrelated': 0 , 'agree':1, 'disagree':2, 'discuss':3}
    for stance in data.stances:
        if(isTest):
            head.append(preprocess(stance['Headline'],head_stop,head_summary))
            body.append(preprocess(data.articles[int(stance['Body ID'])],body_stop,body_summary))
            y.append(category_dict[stance['Stance']])
            continue
        
        if stance['Body ID'] not in ids:
            continue
        
        head.append(preprocess(stance['Headline'],head_stop,head_summary))
        body.append(preprocess(data.articles[int(stance['Body ID'])],body_stop,body_summary))
        y.append(category_dict[stance['Stance']])
    
    head = t.texts_to_sequences(head)
    body = t.texts_to_sequences(body)
    head = pad_sequences(head,maxlen = max_head,padding = 'post')
    body = pad_sequences(body,maxlen = max_body,padding = 'post')
    y_cat = np.zeros((len(y),NUM_CLASSES))
    y_cat = to_categorical(y, num_classes=NUM_CLASSES)
    return head,body,y_cat

def create_labels(data):

#     Usage
#     y_train = create_labels(train_dataset)
#     y_test = create_labels(test_dataset)

    from keras.utils.np_utils import to_categorical
    category_dict = {'unrelated': 0 , 'agree':1, 'disagree':2, 'discuss':3}
    y = list()
    NUM_CLASSES = 4
    for stance in data.stances:
        y.append(category_dict[stance['Stance']])

    y_cat = np.zeros((len(y),NUM_CLASSES))
    y_cat = to_categorical(y, num_classes=NUM_CLASSES)
    return y_cat
'''

In [ ]:
train_body.shape

In [ ]:
embedding_matrix.shape

In [ ]:
max_head = 100
max_body =300
EMBEDDING_DIM = 300

In [ ]:
#from keras.layers import SpatialDropout1D
head_input = Input(shape=(max_head,), dtype='int32', name='head_input')
body_input = Input(shape=(max_body,), dtype='int32', name='body_input')
shared_embed = Embedding(len(tokenizer.word_index)+1 ,EMBEDDING_DIM,weights=[embedding_matrix],trainable=False)
head_embed = shared_embed(head_input)
body_embed = shared_embed(body_input)
head_1d = Conv1D(200,3,padding='same',activation='relu')
body_1d = Conv1D(200,3,padding='same',activation='relu')
# we use max pooling:
shared_lstm = Bidirectional(LSTM(100,dropout=0.2, recurrent_dropout=0.2, name='head_lstm'))
head_conv = head_1d(head_embed)
body_conv = body_1d(body_embed)
head_conv = SpatialDropout1D(0.35)(head_conv)
body_conv = SpatialDropout1D(0.35)(body_conv)
head_lstm = shared_lstm(head_conv)
body_lstm = shared_lstm(body_conv)
dot_layer = dot([head_lstm,body_lstm],axes = 1, normalize=True)

conc = concatenate([head_lstm,body_lstm,dot_layer])
#print(conc)
dense = Dense(100,activation='relu')(conc)
dense = Dropout(0.3)(dense)
dense = Dense(4,activation='softmax')(dense)
model = Model(inputs=[head_input,body_input], outputs=[dense])
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])
print(model.summary())
#SVG(model_to_dot(model).create(prog='dot', format='svg'))
#visualize_model(model)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='test.png', show_shapes=True)

In [ ]:
#
import matplotlib.pyplot as plt
#for i in range(5):
history = model.fit([train_text,train_body],[y_train],validation_data = ([test_text,test_body],y_test),epochs=5, batch_size=64,verbose = True)
#history = model.fit([train_text,train_body],[y_train],epochs=1, batch_size=64,verbose = True)
evaluate_answer(model,[test_text,test_body],y_test)
#evaluate_answer(model,[train_text,train_body],y_train)
#print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("acc.png")
plt.show()
# summarize history for loss 
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("loss.png")
plt.show()

Train on 44974 samples, validate on 4998 samples
Epoch 1/5
44974/44974 [==============================] - 2072s 46ms/step - loss: 0.2750 - acc: 0.8852 - val_loss: 0.1884 - val_acc: 0.9258
Epoch 2/5
22592/44974 [==============>...............] - ETA: 17:07 - loss: 0.2463 - acc: 0.9006

In [24]:
from sklearn.model_selection import train_test_split
train_text,test_text,train_body,test_body,y_train,y_test = train_test_split(train_text,train_body,y_train, test_size=0.1, random_state=42)

In [ ]:
evaluate_answer(model,[test_text,test_body],y_test)

In [ ]:
evaluate_answer(model,[train_text,train_body],y_train)

In [23]:
y_train.shape

(49972, 4)